# Генерация эмоциональной музыки

Здесь попробуем генерить музыку с заранее заданной эмоцией, используя предобученную на LMD GPT2 из статьи https://arxiv.org/abs/2008.06048 и идею условной генерации текста из статьи https://arxiv.org/abs/2011.04000

In [1]:
import os

import json

import pandas as pd
import numpy as np

from operator import add

import torch
import torch.nn.functional as F
from torch.autograd import Variable

from transformers import GPT2Config, GPT2LMHeadModel

from tqdm import trange

from miditok import REMI

In [2]:
# Зададим некоторые констатные значения
SMALL_CONST = 1e-15

# файл, где хранятся списки тактов по эмоциям
BAG_OF_BARS_FILE = os.path.join('emotion_classification', 'linear_clf', 'bar_weights.json')
# Список возможных эмоций
EMOTIONS = {'0': 'cheerful',
            '1': 'tense',
            '2': 'bizarre'}
# эмоция, с которой хотим генерить музыку
EMOTION = EMOTIONS['1']
device = "cuda" if torch.cuda.is_available() and not no_cuda else "cpu"

# Verbosity levels
QUIET = 0
REGULAR = 1
VERBOSE = 2
VERY_VERBOSE = 3

In [3]:
# инициализируем конфиг для GPT2 используя параметры модели из статьи MMM
gpt2config = GPT2Config(vocab_size=647,
                        n_embd=512,
                        n_head=8,
                        n_positions=2048,
                        n_layer=6,
                        output_hidden_states=True
                       )

model = GPT2LMHeadModel(gpt2config)
# Загружаем веса модели
model.load_state_dict(torch.jit.load('models/model.pt').state_dict())
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(647, 512)
    (wpe): Embedding(2048, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropo

In [4]:
# Замораживаем веса
for param in model.parameters():
    param.requires_grad = False

In [5]:
# Загрузим словарь токен - индекс токена для предобученной модели из статьи https://arxiv.org/abs/2008.06048
with open(os.path.join('models', 'tokenizer.json'), 'r') as hfile:
    tokenizer = json.load(hfile)
tokenizer

{'PIECE_START = 0': 0,
 'PIECE_START = 1': 1,
 'NUM_BARS = 4': 2,
 'NUM_BARS = 8': 3,
 'BAR = 0': 4,
 'BAR_END = 0': 5,
 'TIME_SIGNATURE = 1/2': 6,
 'TIME_SIGNATURE = 1/4': 7,
 'TIME_SIGNATURE = 1/8': 8,
 'TIME_SIGNATURE = 1/16': 9,
 'TIME_SIGNATURE = 2/1': 10,
 'TIME_SIGNATURE = 2/2': 11,
 'TIME_SIGNATURE = 2/4': 12,
 'TIME_SIGNATURE = 2/8': 13,
 'TIME_SIGNATURE = 3/1': 14,
 'TIME_SIGNATURE = 3/2': 15,
 'TIME_SIGNATURE = 3/4': 16,
 'TIME_SIGNATURE = 3/8': 17,
 'TIME_SIGNATURE = 4/2': 18,
 'TIME_SIGNATURE = 4/4': 19,
 'TIME_SIGNATURE = 4/8': 20,
 'TIME_SIGNATURE = 5/4': 21,
 'TIME_SIGNATURE = 5/8': 22,
 'TIME_SIGNATURE = 6/2': 23,
 'TIME_SIGNATURE = 6/4': 24,
 'TIME_SIGNATURE = 6/8': 25,
 'TIME_SIGNATURE = 7/4': 26,
 'TIME_SIGNATURE = 7/8': 27,
 'TIME_SIGNATURE = 8/4': 28,
 'TIME_SIGNATURE = 8/8': 29,
 'TIME_SIGNATURE = 9/4': 30,
 'TIME_SIGNATURE = 9/8': 31,
 'TIME_SIGNATURE = 9/16': 32,
 'TIME_SIGNATURE = 10/4': 33,
 'TIME_SIGNATURE = 10/8': 34,
 'TIME_SIGNATURE = 11/8': 35,
 'TIME_SI

In [6]:
# reverse dict for tokens (id:item)
tokenizer_reverse = {val:key for key, val in tokenizer.items()}

In [7]:
generated = model.generate(torch.LongTensor([[0, 2]]), max_length=50, temperature=0.9, top_k=8, )
for idx in generated[0]:
    print(idx.item(), tokenizer_reverse[idx.item()])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0 PIECE_START = 0
2 NUM_BARS = 4
42 TRACK = 10
45 INSTRUMENT = 2
585 MIN_POLYPHONY = 0
595 MAX_POLYPHONY = 0
575 MIN_NOTE_DURATION = 2
582 MAX_NOTE_DURATION = 3
4 BAR = 0
19 TIME_SIGNATURE = 4/4
639 VELOCITY_LEVEL = 99
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
390 TIME_ABSOLUTE = 12
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
402 TIME_ABSOLUTE = 24
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
414 TIME_ABSOLUTE = 36
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
5 BAR_END = 0
4 BAR = 0
19 TIME_SIGNATURE = 4/4
639 VELOCITY_LEVEL = 99
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
390 TIME_ABSOLUTE = 12
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
402 TIME_ABSOLUTE = 24
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
414 TIME_ABSOLUTE = 36
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
5 BAR_END = 0
4 BAR = 0
19 TIME_SIGNATURE = 4/4
639 VELOCITY_LEVEL = 99
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
390 TIME_ABSOLUTE = 12
219 NOTE_ONSET = 65
293 NOTE_DURATION = 11
402 TIME_ABSOLUTE = 24
219 NOTE_ONSET = 65
293

In [8]:
# подготовим контекст
# Так как мы генерим без затравок, то в качестве контекста будем использовать токен начала пьесы PIECE_START
context = [tokenizer['PIECE_START = 0']]
# преобразуем его в тензор и добавим размерностей
context_t = torch.tensor(context, device=device, dtype=torch.long)
while len(context_t.shape) < 2:
    context_t = context_t.unsqueeze(0)
# в переменной output_so_far хранятся сгенерированные токены
output_so_far = context_t

output_so_far

tensor([[0]])

В подход, описанном в статье https://arxiv.org/abs/2011.04000 для контроля за генерацией используются списки слов. А у каждого слова есть свой токен. Следовательно, после каждого сгенерированного слова считается лосс и корректируется выход модели.

В нашем случае есть списки тактов по эмоциям. Значит корректировать выход модели нужно будет только после генерации целого такта.

In [9]:
# def get_affect_words_and_int(emotion):
#     """
#     Нужно получить такты, соответствующие эмоциям
#     :param emotion: конкретная эмоция, такты для которой ищем
#     :return:
#     """
#     bag_of_bars = pd.read_excel(BAG_OF_BARS_FILE, index_col=0)
#     bag_of_bars = bag_of_bars.sort_values(by=emotion)

#     bars = bag_of_bars.tail(1000)

#     return list(bars.index), list(bars[emotion].values)

In [10]:
def get_affect_words_and_int(emotion):
    """
    Нужно получить такты, соответствующие эмоциям
    :param emotion: конкретная эмоция, такты для которой ищем
    :return:
    """
    with open(BAG_OF_BARS_FILE, 'r') as hfile:
        bar_weights = json.load(hfile)
    
    bar_w_df = pd.DataFrame.from_dict(bar_weights).T.rename(columns=EMOTIONS)
    bar_w_df.sort_values(by=emotion, ascending=False, inplace=True)

    bars = bar_w_df.tail(1000)

    return list(bars.index), list(bars[emotion].values)

In [11]:
affect_bars, affect_int = get_affect_words_and_int(EMOTION)

**Проблема:** токенизация в этих списках тактов отличается от токенизации, используемой в модели. 
**Возможное решение:** Надо найти способ "перевести" токены с языка REMI на язык MMM либо переделать классификацию, используя токенизацию MMM. Последний вариант предпочтительнее, так как проще.

Токенизация в mmm_api происходит на языке C, поэтому проблематично извлечь алгоритм оттуда

**Ещё одно возможное решение:** перевести такты, закодированные REMI в миди и их токенизировать с помощью MMM токенизатора

In [12]:
# remi_bars = list(map(lambda elem: elem[-1] + elem[:-2], affect_bars))

In [13]:
# tokenizer = REMI()

In [14]:
# tok_vocab = tokenizer.vocab
# tok_vocab = {key.lower():val for key, val in tok_vocab.items()}

In [15]:
# for index, bar in enumerate(remi_bars):
#     bar_tokens = [tok_vocab[token] for token in bar.split(' ')]
#     midi = tokenizer.tokens_to_midi([bar_tokens])
#     midi.dump(os.path.join('data', 'emotion_bars', EMOTION, f'bar_{index}.mid'))
    

Токенизатор не может закодировать midi, полученные из REMI, значит будем переделывать классификацию.

Классификацию переделали с использованием энкодера из MMM_API. Теперь можно продолжать эксперементы с инференсом

In [16]:
# условие на случай, если захотим подать еще и затравку

uncond = True

if uncond:
    tokenized_cond_text = [tokenizer['PIECE_START = 0'], tokenizer['NUM_BARS = 8']]
else:
    tokenized_cond_text = [tokenizer['PIECE_START = 0'], tokenizer['NUM_BARS = 8']]

In [17]:
# replace tokens in bag of bars with their indices
bob_indices_affect = []  # bob - Bag Of Bars

for bar in affect_bars:
    bob_indices_affect.append([tokenizer[item.upper()] for item in bar.strip().split(', ')])
    
# bob_indices_affect = [bob_indices_affect]
bob_indices_affect

[[4, 19, 380, 631, 216, 329, 5],
 [4,
  19,
  616,
  192,
  381,
  192,
  384,
  192,
  387,
  192,
  390,
  192,
  393,
  192,
  396,
  192,
  399,
  192,
  402,
  192,
  405,
  192,
  408,
  192,
  411,
  192,
  414,
  192,
  417,
  192,
  420,
  192,
  423,
  192,
  5],
 [4,
  19,
  635,
  186,
  287,
  384,
  186,
  287,
  390,
  186,
  287,
  396,
  186,
  287,
  402,
  186,
  287,
  408,
  186,
  287,
  414,
  186,
  287,
  420,
  186,
  287,
  5],
 [4,
  19,
  379,
  631,
  230,
  287,
  385,
  230,
  287,
  391,
  230,
  287,
  397,
  230,
  287,
  403,
  230,
  287,
  409,
  230,
  287,
  415,
  230,
  287,
  421,
  230,
  287,
  5],
 [4, 19, 631, 218, 287, 5],
 [4,
  19,
  381,
  616,
  192,
  388,
  192,
  394,
  192,
  400,
  192,
  406,
  192,
  412,
  192,
  418,
  192,
  424,
  192,
  5],
 [4, 19, 384, 628, 206, 329, 5],
 [4, 19, 379, 646, 221, 370, 5],
 [4, 19, 380, 646, 225, 369, 5],
 [4, 19, 381, 646, 216, 369, 5],
 [4, 19, 628, 216, 329, 5],
 [4, 19, 384, 631, 209, 3

In [18]:
def build_bows_one_hot_vectors_aff(bow_indices, affect_int, tokenizer, device='cuda'):
    if bow_indices is None or affect_int is None:
        return None, None

    one_hot_bows_vectors = []
    print(np.array(bow_indices).shape)
    
    result_affect_int = []
    
    for index, single_bow in enumerate(bow_indices):
#         print(single_bow)
        zipped = [[single_bow[i], affect_int[index]] for i in range(len(single_bow))]
#         print(zipped)
        single_bow_int = list(filter(lambda x: (x[0] in range(154, 378)) and (len(x) >= 1), zipped))
        print(single_bow_int)
        print()
        
        if not single_bow_int:
            continue

        single_bow = [single_bow_int[i][0] for i in range(len(single_bow_int)) ]
        affect_ints = [single_bow_int[i][1] for i in range(len(single_bow_int)) ]
        # print(single_bow, affect_ints)
        # print(len(single_bow), len(affect_ints))
        single_bow = torch.tensor([single_bow]).to(device)
#         print(single_bow)
        num_words = single_bow.shape[0]
#         print(num_words)
        one_hot_bow = torch.zeros(num_words, len(tokenizer)).to(device)
#         print(one_hot_bow.shape)
        one_hot_bow.scatter_(1, single_bow, 1)
        one_hot_bows_vectors.append(one_hot_bow)
        result_affect_int.append(affect_ints[0])
        
    return one_hot_bows_vectors, result_affect_int

In [19]:
affect_bars, affect_int = get_affect_words_and_int(EMOTION)

bow_indices_affect = bob_indices_affect

one_hot_bows_affect, affect_int = build_bows_one_hot_vectors_aff(bow_indices_affect, affect_int, tokenizer, device)

(161,)
[[216, 32.61837731220355], [329, 32.61837731220355]]

[[192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351], [192, 28.60448728402351]]

[[186, 28.095127467161426], [287, 28.095127467161426], [186, 28.095127467161426], [287, 28.095127467161426], [186, 28.095127467161426], [287, 28.095127467161426], [186, 28.095127467161426], [287, 28.095127467161426], [186, 28.095127467161426], [287, 28.095127467161426], [186, 28.095127467161426], [287, 28.095127467161426], [186, 28.095127467161426], [287, 28.095127467161426], [186, 28.095127467161426], [287, 28.095127467161426]]

[[230, 27.816581384940115], [287, 27.816581384940115], [230, 27.816581384940115], [287,

/var/folders/nt/zczh006s6qbfkmy1ywwcw_1mqbxly0/T/ipykernel_75122/2361642276.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(bow_indices).shape)


In [20]:
# affect_bars, affect_int = get_affect_words_and_int(EMOTION)


In [21]:
len(one_hot_bows_affect)

160

In [22]:
len(affect_int)

160

In [23]:
new_affect_int = list(map(lambda elem: elem - min(affect_int) + SMALL_CONST, affect_int))
new_affect_int = new_affect_int / max(new_affect_int)
new_affect_int

array([1.00000000e+00, 9.41351827e-01, 9.33909415e-01, 9.29839493e-01,
       9.29393703e-01, 8.78585488e-01, 8.52390898e-01, 8.34737234e-01,
       8.34737234e-01, 8.34737234e-01, 8.33736203e-01, 8.21896793e-01,
       8.15581776e-01, 8.07383753e-01, 7.91403763e-01, 7.86312587e-01,
       7.75315151e-01, 7.63248361e-01, 7.58779681e-01, 7.55236587e-01,
       7.50619764e-01, 7.47842729e-01, 7.46188815e-01, 7.46154691e-01,
       7.46154691e-01, 7.29711950e-01, 7.29661784e-01, 7.02512452e-01,
       6.95593190e-01, 6.87447195e-01, 6.85228509e-01, 6.81788053e-01,
       6.80354809e-01, 6.71832853e-01, 6.70372296e-01, 6.68643645e-01,
       6.66411272e-01, 6.64903748e-01, 6.60302675e-01, 6.60187121e-01,
       6.54870429e-01, 6.54319233e-01, 6.53236018e-01, 6.50973636e-01,
       6.49644322e-01, 6.47662131e-01, 6.46654678e-01, 6.36082676e-01,
       6.30567144e-01, 6.27684868e-01, 6.25864831e-01, 6.18423018e-01,
       6.16913407e-01, 6.15473532e-01, 6.14175821e-01, 6.14175821e-01,
      

In [24]:
one_hot_bows_affect[0].shape

torch.Size([1, 647])

In [25]:
def to_var(x, requires_grad=False, volatile=False, device='cuda'):
    if torch.cuda.is_available() and device == 'cuda':
        x = x.cuda()
    elif device != 'cuda':
        x = x.to(device)
    return Variable(x, requires_grad=requires_grad, volatile=volatile)

def gaussian(x, mu, sig):
    x = np.array(x)
    return list(np.exp(-0.5*((x-mu)/sig)**2)/(sig*(2*np.pi)**0.5))

def top_k_filter(logits, k, probs=False):
    """
    Masks everything but the k top entries as -infinity (1e10).
    Used to mask logits such that e^-infinity -> 0 won't contribute to the
    sum of the denominator.
    """
    if k == 0:
        return logits
    else:
        values = torch.topk(logits, k)[0]
        batch_mins = values[:, -1].view(-1, 1).expand_as(logits)
        if probs:
            return torch.where(logits < batch_mins,
                               torch.ones_like(logits) * 0.0, logits)
        return torch.where(logits < batch_mins,
                           torch.ones_like(logits) * -BIG_CONST,
                           logits)

In [26]:
verbosity_level = VERBOSE
length = 500

grad_norms = None
last = None
unpert_discrim_loss = 0
loss_in_time = []

if verbosity_level >= VERBOSE:
    range_func = trange(length, ascii=True)
else:
    range_func = range(length)
count = 0
int_score = 0

  0%|                                                                                                                                                                                      | 0/500 [00:00<?, ?it/s]

In [27]:
# trying to generate music

output_so_far = None
context = tokenized_cond_text

if context:
    context_t = torch.tensor(context, device=device, dtype=torch.long)
    while len(context_t.shape) < 2:
        context_t = context_t.unsqueeze(0)
    output_so_far = context_t

In [28]:
output_so_far

tensor([[0, 3]])

In [29]:
past = None
grad_length = 10000
stepsize = 0.02
perturb = True
decay = False
window_length = 5
num_iterations = 3
temperature = 0.8

BOW_AFFECT = 4
PPLM_BOW = 1

loss_type = BOW_AFFECT

knob = 0.7
affect_weight = 1
kl_scale = 0.0
gamma = 1.5
classifier = None
gm_scale = 0.9
top_k = 10
sample = True

for i in range_func:
# for i in range(length):
    # if count == 3:
    #   break
    # Get past/probs for current output, except for last word
    # Note that GPT takes 2 inputs: past + current_token

    # run model forward to obtain unperturbed
    if past is None and output_so_far is not None:
        last = output_so_far[:, -1:]
        if output_so_far.shape[1] > 1:
            past = model(output_so_far[:, :-1])['past_key_values']
    
    print(output_so_far)
#     print(past)

    model_out = model(output_so_far)
#     print(model_out)
    unpert_logits = model_out['logits']
    unpert_past = model_out['past_key_values']
    unpert_all_hidden = model_out['hidden_states']
    unpert_last_hidden = unpert_all_hidden[-1]
    
    logits = unpert_logits[:, -1, :]
    probs = F.softmax(logits, dim=-1)

    last = torch.multinomial(probs, num_samples=1)
#     print("LAST")
#     print(last)
    
    # check if we are abowe grad max length
    if i >= grad_length:
        current_stepsize = stepsize * 0
    else:
        current_stepsize = stepsize

    # modify the past if necessary
    if not perturb or num_iterations == 0 or last[0] not in range(154, 378):
        pert_past = past
        
        output_so_far = (
            last if output_so_far is None
            else torch.cat((output_so_far, last), dim=1)
        )
    
    else:
        accumulated_hidden = unpert_last_hidden[:, :-1, :]
        accumulated_hidden = torch.sum(accumulated_hidden, dim=1)
        
        if past is not None:
            # start of func perturb_past
            # Generate inital perturbed past
            grad_accumulator = [
                (np.zeros(p[0].shape).astype("float32"), 
                 np.zeros(p[1].shape).astype("float32"))
                for p in past
            ]
            
            if accumulated_hidden is None:
                accumulated_hidden = 0
                
            if decay:
                decay_mask = torch.arange(
                    0.,
                    1.0 + SMALL_CONST,
                    1.0 / (window_length)
                )[1:]
            else:
                decay_mask = 1.0
            
#             print(past[0][0].shape)
            _, _, curr_length, _ = past[0][0].shape
            
            if curr_length > window_length and window_length > 0:
                ones_key_val_shape = (
                        tuple([2])
                        + tuple(past[0][0].shape[:-2])
                        + tuple([window_length])
                        + tuple(past[0][0].shape[-1:])
                )

                zeros_key_val_shape = (
                        tuple([2])
                        + tuple(past[0][0].shape[:-2])
                        + tuple([curr_length - window_length])
                        + tuple(past[0][0].shape[-1:])
                )
                print(ones_key_val_shape)
                ones_mask = torch.ones(ones_key_val_shape)
                ones_mask = decay_mask * ones_mask.permute(0, 1, 2, 4, 3)
                ones_mask = ones_mask.permute(0, 1, 2, 4, 3)

                window_mask = torch.cat(
                    (ones_mask, torch.zeros(zeros_key_val_shape)),
                    dim=-2
                ).to(device)
                print(window_mask.shape)
            else:
                window_mask = (torch.ones_like(past[0][0]).to(device), 
                               torch.ones_like(past[0][1]).to(device))

            # accumulate perturbations for num_iterations
            loss_per_iter = []
            new_accumulated_hidden = None
            
            for i in range(num_iterations):
                if verbosity_level >= VERBOSE:
                    print("Iteration ", i + 1)
                curr_perturbation = [
                    (to_var(torch.from_numpy(p_[0]), requires_grad=True, device=device),
                     to_var(torch.from_numpy(p_[1]), requires_grad=True, device=device))
                    for p_ in grad_accumulator
                ]
                # Compute hidden using perturbed past
                
                perturbed_past = tuple(tuple(map(add, curr_perturb, past_i)) for curr_perturb, past_i in zip(curr_perturbation, past))
    
                _, _, curr_length, _ = curr_perturbation[0][0].shape
                
#                 print(len(perturbed_past))
#                 print(len(perturbed_past[0]))
                
                model_out = model(last, past_key_values=perturbed_past)
                
                all_logits = model_out['logits']
                all_hidden = model_out['hidden_states']
                
#                 all_logits, _, all_hidden = model(last, past_key_values=perturbed_past)
                hidden = all_hidden[-1]
                new_accumulated_hidden = accumulated_hidden + torch.sum(
                    hidden,
                    dim=1
                ).detach()
        
                logits = all_logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                
#                 print('PROBS')
#                 print(probs.shape)
#                 print(probs)
                
                loss = 0.0
                loss_list = []
                
                if loss_type == BOW_AFFECT:
                    for one_hot_bow, aff_int in zip(one_hot_bows_affect, new_affect_int):
                        bow_logits = torch.mm(probs, torch.t(one_hot_bow))

#                             print(bow_logits.shape)
#                         print(bow_logits)
#                             print(torch.t(
#                                 torch.FloatTensor(gaussian([aff_int], knob, .1))))

                        bow_loss = -torch.log(torch.matmul(bow_logits, torch.t(
                            torch.FloatTensor(gaussian([aff_int], knob, .1)).to(
                                device))))
#                         print(bow_loss)

                        loss += affect_weight * bow_loss[0]
                        loss_list.append(bow_loss)
                    
                kl_loss = 0.0
                if kl_scale > 0.0:
                    unpert_probs = F.softmax(unpert_logits[:, -1, :], dim=-1)
                    unpert_probs = (
                            unpert_probs + SMALL_CONST *
                            (unpert_probs <= SMALL_CONST).float().to(device).detach()
                    )
                    correction = SMALL_CONST * (probs <= SMALL_CONST).float().to(
                        device).detach()
                    corrected_probs = probs + correction.detach()
                    kl_loss = kl_scale * (
                        (corrected_probs * (corrected_probs / unpert_probs).log()).sum()
                    )
                    if verbosity_level >= VERY_VERBOSE:
                        print(' kl_loss', kl_loss.data.cpu().numpy())
                    loss += kl_loss
                
                loss_per_iter.append(loss.data.cpu().numpy())
                print(loss)
                # compute gradients
                loss.backward()
        
                # calculate gradient norms
                if grad_norms is not None and loss_type == PPLM_BOW:
                    grad_norms = [
                        torch.max(grad_norms[index], torch.norm(p_.grad * window_mask))
                        for index, p_ in enumerate(curr_perturbation)
                    ]
                else:
                    grad_norms = [
                        (torch.norm(p_[0].grad * window_mask[0]) + SMALL_CONST,
                         torch.norm(p_[1].grad * window_mask[1]) + SMALL_CONST)
                        for index, p_ in enumerate(curr_perturbation)
                    ]
                
                # normalize gradients
                grad = [
                    
                    (-stepsize * (p_[0].grad * window_mask[0] / grad_norms[
                        index][0] ** gamma).data.cpu().numpy(), 
                     -stepsize * (p_[1].grad * window_mask[1] / grad_norms[
                        index][1] ** gamma).data.cpu().numpy())
                    for index, p_ in enumerate(curr_perturbation)
                ]
                
                # accumulate gradient
#                 grad_accumulator = list(map(add, grad, grad_accumulator))
                grad_accumulator = tuple(tuple(map(add, curr_perturb, past_i)) for curr_perturb, past_i in zip(grad, grad_accumulator))
#                 print(len(grad_accumulator))
#                 print(len(grad_accumulator[0]))
                
                # reset gradients, just to make sure
                for p_ in curr_perturbation:
                    p_[0].grad.data.zero_()
                    p_[1].grad.data.zero_()
                    
                # removing past from the graph
                new_past = []
                for p_ in past:
                    new_past.append((p_[0].detach(), p_[1].detach()))
                past = new_past
                
            # apply the accumulated perturbations to the past
            grad_accumulator = [
                (to_var(torch.from_numpy(p_[0]), requires_grad=True, device=device),
                 to_var(torch.from_numpy(p_[1]), requires_grad=True, device=device))
                for p_ in grad_accumulator
            ]
#             pert_past = list(map(add, past, grad_accumulator))
            pert_past = tuple(tuple(map(add, curr_perturb, past_i)) for curr_perturb, past_i in zip(grad_accumulator, past))
                            
        else:
            pert_past = past
            print('pert_past=past')
        
#         pert_logits, past, pert_all_hidden = model(last, past_key_values=pert_past)
        model_out = model(last, past_key_values=pert_past)
        pert_logits = model_out['logits']
        past = model_out['past_key_values']
        pert_all_hidden = model_out['hidden_states']
        
        pert_logits = pert_logits[:, -1, :] / temperature  # + SMALL_CONST
        pert_probs = F.softmax(pert_logits, dim=-1)
        
        if classifier is not None:
            ce_loss = torch.nn.CrossEntropyLoss()
            prediction = classifier(torch.mean(unpert_last_hidden, dim=1))
            label = torch.tensor([class_label], device=device,
                                 dtype=torch.long)
            unpert_discrim_loss = ce_loss(prediction, label)
            if verbosity_level >= VERBOSE:
                print(
                    "unperturbed discrim loss",
                    unpert_discrim_loss.data.cpu().numpy()
                )
        else:
            unpert_discrim_loss = 0
            
        # Fuse the modified model and original model
        if perturb:

            unpert_probs = F.softmax(unpert_logits[:, -1, :], dim=-1)

            pert_probs = ((pert_probs ** gm_scale) * (
                    unpert_probs ** (1 - gm_scale)))  # + SMALL_CONST
            pert_probs = top_k_filter(pert_probs, k=top_k,
                                      probs=True)  # + SMALL_CONST

            # rescale
            if torch.sum(pert_probs) <= 1:
                pert_probs = pert_probs / torch.sum(pert_probs)

        else:
            pert_logits = top_k_filter(pert_logits, k=top_k)  # + SMALL_CONST
            pert_probs = F.softmax(pert_logits, dim=-1)
        
        # sample or greedy
        if sample:
            last = torch.multinomial(pert_probs, num_samples=1)
            # print('pert_prob, last ', pert_probs, last)

        else:
            _, last = torch.topk(pert_probs, k=1, dim=-1)
            
        # update context/output_so_far appending the new token
        output_so_far = (
            last if output_so_far is None
            else torch.cat((output_so_far, last), dim=1)
        )
        

  2%|###1                                                                                                                                                                          | 9/500 [00:02<01:25,  5.74it/s]

tensor([[0, 3]])
tensor([[ 0,  3, 42]])
tensor([[ 0,  3, 42, 45]])
tensor([[  0,   3,  42,  45, 585]])
tensor([[  0,   3,  42,  45, 585, 597]])
tensor([[  0,   3,  42,  45, 585, 597, 575]])
tensor([[  0,   3,  42,  45, 585, 597, 575, 582]])
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4]])
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11]])
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11, 634]])
Iteration  1
tensor(967.2258, grad_fn=<AddBackward0>)
Iteration  2
tensor(965.4296, grad_fn=<AddBackward0>)
Iteration  3
tensor(963.6726, grad_fn=<AddBackward0>)
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11, 634, 390]])
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11, 634, 390, 634]])
Iteration  1
tensor(1439.8795, grad_fn=<AddBackward0>)
Iteration  2
tensor(1438.0583, grad_fn=<AddBackward0>)
Iteration  3
tensor(1436.2377, grad_fn=<AddBackward0>)
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11, 634, 390, 634, 208]])
Iteration  1
tensor(1

  3%|####8                                                                                                                                                                        | 14/500 [00:02<01:11,  6.78it/s]

Iteration  2
tensor(1647.2479, grad_fn=<AddBackward0>)
Iteration  3
tensor(1645.6962, grad_fn=<AddBackward0>)
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11, 634, 390, 634, 208,
         211, 211]])
Iteration  1
tensor(1668.9601, grad_fn=<AddBackward0>)
Iteration  2
tensor(1667.3728, grad_fn=<AddBackward0>)
Iteration  3
tensor(1665.7871, grad_fn=<AddBackward0>)


  3%|#####8                                                                                                                                                                       | 17/500 [00:02<01:04,  7.50it/s]

...

(2, 1, 8, 5, 64)
torch.Size([2, 1, 8, 241, 64])
Iteration  1
tensor(1293.1754, grad_fn=<AddBackward0>)
Iteration  2
tensor(1292.2811, grad_fn=<AddBackward0>)
Iteration  3
tensor(1291.3866, grad_fn=<AddBackward0>)
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11, 634, 390, 634, 208,
         211, 211, 384, 633, 293, 396, 293, 408, 293, 414, 634, 293,   5,   4,
          11, 633, 293, 390, 287, 293, 408, 632, 282, 414, 633, 293,   5,   4,
          11, 390, 633, 204, 207, 207, 204, 390, 634, 293, 633, 293, 408, 293,
         634, 293, 633, 293, 414, 634, 287, 420, 293, 329,   5,   4,  11, 384,
         633, 293,   5,   4,  11, 633, 293, 632, 293, 634, 216, 390, 632, 293,
         633, 206, 218, 216, 384, 293, 390, 293, 396, 293, 402, 634, 287, 408,
         633, 204, 207, 206, 204, 207, 208, 208, 204, 204, 204,   5,   4,  11,
         633, 293, 634, 293, 633, 287, 384, 293, 396, 632, 287, 402, 633, 293,
         634, 299, 420, 287,   5,   4,  11, 632, 293, 634, 293, 390, 293, 40

100%|###########################################################################################################################################################################6| 499/500 [02:10<00:00,  1.75it/s]

(2, 1, 8, 5, 64)
torch.Size([2, 1, 8, 242, 64])
Iteration  1
tensor(1066.2181, grad_fn=<AddBackward0>)
Iteration  2
tensor(1065.2990, grad_fn=<AddBackward0>)
Iteration  3
tensor(1064.3794, grad_fn=<AddBackward0>)
tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11, 634, 390, 634, 208,
         211, 211, 384, 633, 293, 396, 293, 408, 293, 414, 634, 293,   5,   4,
          11, 633, 293, 390, 287, 293, 408, 632, 282, 414, 633, 293,   5,   4,
          11, 390, 633, 204, 207, 207, 204, 390, 634, 293, 633, 293, 408, 293,
         634, 293, 633, 293, 414, 634, 287, 420, 293, 329,   5,   4,  11, 384,
         633, 293,   5,   4,  11, 633, 293, 632, 293, 634, 216, 390, 632, 293,
         633, 206, 218, 216, 384, 293, 390, 293, 396, 293, 402, 634, 287, 408,
         633, 204, 207, 206, 204, 207, 208, 208, 204, 204, 204,   5,   4,  11,
         633, 293, 634, 293, 633, 287, 384, 293, 396, 632, 287, 402, 633, 293,
         634, 299, 420, 287,   5,   4,  11, 632, 293, 634, 293, 390, 293, 40

100%|############################################################################################################################################################################| 500/500 [02:11<00:00,  3.81it/s]

(2, 1, 8, 5, 64)
torch.Size([2, 1, 8, 243, 64])
Iteration  1
tensor(1534.6598, grad_fn=<AddBackward0>)
Iteration  2
tensor(1533.5447, grad_fn=<AddBackward0>)
Iteration  3
tensor(1532.4286, grad_fn=<AddBackward0>)


In [30]:
output_so_far.shape

torch.Size([1, 502])

In [31]:
output_so_far

tensor([[  0,   3,  42,  45, 585, 597, 575, 582,   4,  11, 634, 390, 634, 208,
         211, 211, 384, 633, 293, 396, 293, 408, 293, 414, 634, 293,   5,   4,
          11, 633, 293, 390, 287, 293, 408, 632, 282, 414, 633, 293,   5,   4,
          11, 390, 633, 204, 207, 207, 204, 390, 634, 293, 633, 293, 408, 293,
         634, 293, 633, 293, 414, 634, 287, 420, 293, 329,   5,   4,  11, 384,
         633, 293,   5,   4,  11, 633, 293, 632, 293, 634, 216, 390, 632, 293,
         633, 206, 218, 216, 384, 293, 390, 293, 396, 293, 402, 634, 287, 408,
         633, 204, 207, 206, 204, 207, 208, 208, 204, 204, 204,   5,   4,  11,
         633, 293, 634, 293, 633, 287, 384, 293, 396, 632, 287, 402, 633, 293,
         634, 299, 420, 287,   5,   4,  11, 632, 293, 634, 293, 390, 293, 402,
         633, 287, 408, 209, 206, 211, 209, 206, 204, 206, 204, 206, 207, 207,
         207, 293,   5,   4,  11, 634, 293, 390, 633, 293, 402, 634, 293, 414,
         633, 293,   5,  44,  42,  45, 585, 597, 576

In [153]:
for token in output_so_far[0]:
    print(tokenizer_reverse[token.item()])

PIECE_START = 0
NUM_BARS = 4
TRACK = 10
INSTRUMENT = 34
MIN_POLYPHONY = 0
MAX_POLYPHONY = 0
MIN_NOTE_DURATION = 5
MAX_NOTE_DURATION = 5
BAR = 0
TIME_SIGNATURE = 4/4
VELOCITY_LEVEL = 99
NOTE_DURATION = 5
TIME_ABSOLUTE = 6
NOTE_ONSET = 36
NOTE_DURATION = 65
BAR_END = 0
BAR = 0
TIME_SIGNATURE = 4/4
TIME_ABSOLUTE = 24
VELOCITY_LEVEL = 99
NOTE_ONSET = 35
NOTE_DURATION = 47
BAR_END = 0
BAR = 0
TIME_SIGNATURE = 4/4
TIME_ABSOLUTE = 24
VELOCITY_LEVEL = 99
NOTE_ONSET = 35
NOTE_DURATION = 71
BAR_END = 0
BAR = 0
TIME_SIGNATURE = 4/4
BAR_END = 0
TRACK_END = 0
TRACK = 10
INSTRUMENT = 2
MIN_POLYPHONY = 0
MAX_POLYPHONY = 4
MIN_NOTE_DURATION = 2
MAX_NOTE_DURATION = 4
BAR = 0
TIME_SIGNATURE = 4/4
TIME_ABSOLUTE = 6
VELOCITY_LEVEL = 99
NOTE_ONSET = 50
NOTE_DURATION = 5
TIME_ABSOLUTE = 12
NOTE_ONSET = 58
NOTE_DURATION = 23
TIME_ABSOLUTE = 24
NOTE_ONSET = 57
NOTE_DURATION = 11
TIME_ABSOLUTE = 30
NOTE_ONSET = 58
NOTE_DURATION = 11
TIME_ABSOLUTE = 36
NOTE_ONSET = 55
NOTE_DURATION = 35
TIME_ABSOLUTE = 42
NOTE_

In [76]:
pert_past

((tensor([[[[-3.6121, -0.7460, -0.8052,  ..., -0.4343, -0.8173, -0.6579],
            [-3.1073,  0.8341, -1.5847,  ..., -0.4142,  0.1936, -1.6403],
            [-1.1539,  0.1196, -0.5486,  ..., -0.1897, -0.5538, -1.1324],
            ...,
            [-0.8256,  0.5778, -0.6096,  ...,  0.4159,  0.0811, -0.9520],
            [-0.9905, -0.2375,  0.6059,  ...,  0.2181, -1.5774, -1.0783],
            [-2.0661,  0.8114, -0.4450,  ..., -0.3191, -0.5547, -0.5712]],
  
           [[ 0.5673,  0.8013,  2.7591,  ...,  2.0936,  0.5356,  0.9839],
            [ 0.3821, -0.6622,  4.5116,  ...,  2.1856,  0.3906, -0.3804],
            [ 0.0522, -0.4710,  2.0756,  ...,  1.4309,  0.7774,  0.9130],
            ...,
            [-1.5688, -0.7817,  2.0935,  ..., -0.2662, -0.2221, -0.5081],
            [-0.1972, -0.0672,  2.3714,  ...,  1.4996, -0.0337, -0.1113],
            [ 0.2839, -0.0679,  2.1388,  ...,  1.2536,  0.6876, -0.7365]],
  
           [[-0.0982, -0.7451,  1.6304,  ..., -0.0558, -2.0750,  1.736

In [182]:
model(last, past_key_values=perturbed_past)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[-4.1876e+00, -4.8291e+00,  1.5444e+00,  1.7269e+00,  1.2674e+00,
          -2.8787e-01, -1.9594e+00, -1.8490e+00, -3.6825e+00, -2.3843e+00,
          -8.8429e-01, -3.5897e-01,  2.1313e-01, -2.8533e+00, -3.8681e-01,
          -5.5471e-01, -1.3413e-02, -1.5136e+00,  1.0236e-01,  2.3060e+00,
          -2.3151e+00, -7.6952e-01, -2.7972e+00, -1.9081e+00, -3.7542e-01,
          -4.8850e-01, -1.5278e+00, -2.0996e+00, -4.0530e-01, -1.5491e+00,
          -1.2923e+00, -1.4484e+00, -2.1404e+00, -1.4534e+00, -3.1077e+00,
          -3.3687e+00, -1.2493e+00, -6.0852e-01, -4.0528e+00, -3.2829e+00,
          -9.2921e-01, -3.2282e+00,  1.4149e+01,  1.2465e+01,  1.8846e+00,
           3.9209e+00,  1.0921e+00,  2.1183e+00,  1.8832e+00,  1.0812e+00,
           9.9143e-01, -4.9869e-01,  1.1327e+00,  1.7467e+00,  5.7013e-01,
          -7.0625e-01, -2.3397e-01, -4.5974e-01,  2.1390e+00,  1.6232e+00,
           9.4808e-01,  1.2353e+00,  6.7042e-01,

In [178]:
perturb_past = tuple(tuple(map(add, curr_perturb, past_i)) for curr_perturb, past_i in zip(curr_perturbation, past))


In [180]:
len(perturb_past[0])

2

In [139]:
model(output_so_far[:, :-1])['past_key_values'][0][1]

tensor([[[[ 0.6711, -0.0431,  0.0322, -0.1003,  0.1303,  0.0401, -0.4259,
           -0.1421, -0.4042,  0.1043,  0.2052,  0.1887, -0.2424, -0.3224,
            0.2825, -0.0031,  0.5778,  0.0439, -0.0023, -0.1685,  0.1409,
           -0.0300, -0.2547, -0.4937,  0.3660,  0.3420,  0.2700, -0.3454,
            0.2395, -0.0273,  0.0492,  0.5396,  0.1451,  0.3028, -0.3101,
           -0.4488,  0.2750,  0.2073, -0.6259, -0.4431, -0.1293, -0.2258,
           -0.1269,  0.3144, -0.1765,  0.1555, -0.3957, -0.0929, -0.1654,
            0.1099,  0.4094, -0.0284,  0.1988, -0.3360, -0.0473,  0.2106,
           -0.0702, -0.1505, -0.5452,  0.0654, -0.0743,  0.3867,  0.0943,
            0.7922]],

         [[ 0.1598, -0.0240, -0.0221,  0.0821,  0.2434,  0.1491, -0.1292,
            0.0263, -0.1864, -0.3264, -0.1360, -0.0184, -0.0770, -0.1920,
            0.2943, -0.1668,  0.1865, -0.0509,  0.1117, -0.1149,  0.4011,
            0.3865, -0.1905,  0.4107, -0.1525, -0.2647, -0.2140, -0.4953,
            0.0

In [135]:
model(output_so_far[:, :-1])['past_key_values'][3]

(tensor([[[[-4.6959e-01, -1.0059e+00, -3.0761e+00,  1.8824e+00,  1.1027e+00,
            -9.2851e-01, -1.1297e+00, -5.1756e-01, -1.7689e-01,  4.6326e-01,
            -2.6025e+00, -1.0120e+00,  1.5064e+00,  1.6901e+00,  1.0835e+00,
             4.2517e-01, -8.8605e-01, -3.4672e-01,  1.4630e+00, -1.4076e+00,
             4.5802e-01,  1.7408e-02, -9.8819e-02, -5.2309e-01,  7.2204e-01,
            -4.6965e-01,  1.2680e+00, -4.4466e-01, -2.0059e+00,  2.4456e-01,
             5.6571e-01, -3.5213e-01, -1.2932e+00, -5.4734e-01,  6.6009e-01,
            -1.4751e-01, -8.5196e-01,  4.6824e-01, -1.1037e+00, -4.5536e-01,
             3.1028e-01, -2.5350e+00,  3.5584e+00, -7.4055e-01, -1.0445e+00,
            -2.3582e-01,  3.5622e-01,  2.4488e+00, -3.7516e-02,  1.4163e+00,
            -1.2974e-02, -3.1056e-01, -9.4534e-01,  2.6252e+00,  7.8000e-01,
             5.4642e-01, -1.4258e+00, -1.6912e+00, -1.9248e-01,  3.9324e-01,
            -9.8528e-01, -2.0585e-01,  6.8237e-01, -6.7844e-01]],
 
        

In [142]:
for i in range(len(past)):
    for j in range(len(past[i])):
        _, _, curr_length, _ = past[0][0].shape
        print(past[i][j].shape)

torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])
torch.Size([1, 8, 1, 64])


In [ ]:
perturb_past(
                    past,
                    model,
                    last,
                    affect_weight=affect_weight,
                    unpert_past=unpert_past,
                    unpert_logits=unpert_logits,
                    accumulated_hidden=accumulated_hidden,
                    grad_norms=grad_norms,
                    stepsize=current_stepsize,
                    # one_hot_bows_vectors=one_hot_bows_vectors,
                    one_hot_bows_affect=one_hot_bows_affect,
                    affect_int=affect_int,
                    knob=knob,
                    classifier=classifier,
                    class_label=class_label,
                    loss_type=loss_type,
                    num_iterations=num_iterations,
                    horizon_length=horizon_length,
                    window_length=window_length,
                    decay=decay,
                    gamma=gamma,
                    kl_scale=kl_scale,
                    device=device,
                    verbosity_level=verbosity_level
                )

In [ ]:
def perturb_past(
        past,
        model,
        last,
        affect_weight=0.2,
        unpert_past=None,
        unpert_logits=None,
        accumulated_hidden=None,
        grad_norms=None,
        stepsize=0.01,
        one_hot_bows_vectors=None,
        one_hot_bows_affect=None,
        affect_int = None,
        knob = None,
        classifier=None,
        class_label=None,
        loss_type=0,
        num_iterations=3,
        horizon_length=1,
        window_length=0,
        decay=False,
        gamma=1.5,
        kl_scale=0.01,
        device='cuda',
        verbosity_level=REGULAR
):
    # Generate inital perturbed past
    grad_accumulator = [
        (np.zeros(p.shape).astype("float32"))
        for p in past
    ]
    
    if accumulated_hidden is None:
        accumulated_hidden = 0

    if decay:
        decay_mask = torch.arange(
            0.,
            1.0 + SMALL_CONST,
            1.0 / (window_length)
        )[1:]
    else:
        decay_mask = 1.0

    # TODO fix this comment (SUMANTH)
    # Generate a mask is gradient perturbated is based on a past window
    _, _, _, curr_length, _ = past[0].shape

    if curr_length > window_length and window_length > 0:
        ones_key_val_shape = (
                tuple(past[0].shape[:-2])
                + tuple([window_length])
                + tuple(past[0].shape[-1:])
        )

        zeros_key_val_shape = (
                tuple(past[0].shape[:-2])
                + tuple([curr_length - window_length])
                + tuple(past[0].shape[-1:])
        )

        ones_mask = torch.ones(ones_key_val_shape)
        ones_mask = decay_mask * ones_mask.permute(0, 1, 2, 4, 3)
        ones_mask = ones_mask.permute(0, 1, 2, 4, 3)

        window_mask = torch.cat(
            (ones_mask, torch.zeros(zeros_key_val_shape)),
            dim=-2
        ).to(device)
    else:
        window_mask = torch.ones_like(past[0]).to(device)

    # accumulate perturbations for num_iterations
    loss_per_iter = []
    new_accumulated_hidden = None
    for i in range(num_iterations):
        if verbosity_level >= VERBOSE:
            print("Iteration ", i + 1)
        curr_perturbation = [
            to_var(torch.from_numpy(p_), requires_grad=True, device=device)
            for p_ in grad_accumulator
        ]

        # Compute hidden using perturbed past
        perturbed_past = list(map(add, past, curr_perturbation))
        _, _, _, curr_length, _ = curr_perturbation[0].shape
        all_logits, _, all_hidden = model(last, past=perturbed_past)
        hidden = all_hidden[-1]
        new_accumulated_hidden = accumulated_hidden + torch.sum(
            hidden,
            dim=1
        ).detach()
        # TODO: Check the layer-norm consistency of this with trained discriminator (Sumanth)
        logits = all_logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)

        loss = 0.0
        loss_list = []
        if loss_type == PPLM_BOW or loss_type == BOW_AFFECT:
            for one_hot_bow in one_hot_bows_vectors:
                bow_logits = torch.mm(probs, torch.t(one_hot_bow))
                #print(type(bow_logits))
                bow_loss = -torch.log(torch.sum(bow_logits))
                #print(bow_loss)
                loss +=  bow_loss
                loss_list.append(bow_loss)
            if loss_type == BOW_AFFECT:
                for one_hot_bow in one_hot_bows_affect:
                    bow_logits = torch.mm(probs, torch.t(one_hot_bow))
                   # print(bow_logits.size(), torch.FloatTensor(affect_int).size())
                    bow_loss = -torch.log(torch.matmul(bow_logits, torch.t(torch.FloatTensor(gaussian(affect_int, knob, .1)).to(device))))#-torch.log(torch.sum(bow_logits))#
                    # print(bow_loss)
  
                    loss += affect_weight * bow_loss[0]
                    loss_list.append(bow_loss)
            if verbosity_level >= VERY_VERBOSE:
                print(" pplm_bow_loss:", loss.data.cpu().numpy())

        kl_loss = 0.0
        if kl_scale > 0.0:
            unpert_probs = F.softmax(unpert_logits[:, -1, :], dim=-1)
            unpert_probs = (
                    unpert_probs + SMALL_CONST *
                    (unpert_probs <= SMALL_CONST).float().to(device).detach()
            )
            correction = SMALL_CONST * (probs <= SMALL_CONST).float().to(
                device).detach()
            corrected_probs = probs + correction.detach()
            kl_loss = kl_scale * (
                (corrected_probs * (corrected_probs / unpert_probs).log()).sum()
            )
            if verbosity_level >= VERY_VERBOSE:
                print(' kl_loss', kl_loss.data.cpu().numpy())
            loss += kl_loss

        loss_per_iter.append(loss.data.cpu().numpy())
        if verbosity_level >= VERBOSE:
            print(' pplm_loss', (loss - kl_loss).data.cpu().numpy())

        # compute gradients
        loss.backward()

        # calculate gradient norms
        if grad_norms is not None and loss_type == PPLM_BOW:
            grad_norms = [
                torch.max(grad_norms[index], torch.norm(p_.grad * window_mask))
                for index, p_ in enumerate(curr_perturbation)
            ]
        else:
            grad_norms = [
                (torch.norm(p_.grad * window_mask) + SMALL_CONST)
                for index, p_ in enumerate(curr_perturbation)
            ]

        # normalize gradients
        grad = [
            -stepsize *
            (p_.grad * window_mask / grad_norms[
                index] ** gamma).data.cpu().numpy()
            for index, p_ in enumerate(curr_perturbation)
        ]

        # accumulate gradient
        grad_accumulator = list(map(add, grad, grad_accumulator))

        # reset gradients, just to make sure
        for p_ in curr_perturbation:
            p_.grad.data.zero_()

        # removing past from the graph
        new_past = []
        for p_ in past:
            new_past.append(p_.detach())
        past = new_past

    # apply the accumulated perturbations to the past
    grad_accumulator = [
        to_var(torch.from_numpy(p_), requires_grad=True, device=device)
        for p_ in grad_accumulator
    ]
    pert_past = list(map(add, past, grad_accumulator))

    return pert_past, new_accumulated_hidden, grad_norms, loss_per_iter

In [13]:
def load_model():
    # инициализируем конфиг для GPT2 используя параметры модели из статьи MMM
    gpt2config = GPT2Config(vocab_size=647,
                            n_embd=512,
                            n_head=8,
                            n_positions=2048,
                            n_layer=6
                           )

    model = GPT2LMHeadModel(gpt2config)
    # Загружаем веса модели
    model.load_state_dict(torch.jit.load('models/model.pt').state_dict())
    
    with open(os.path.join('models', 'tokenizer.json'), 'r') as hfile:
        tokenizer = json.load(hfile)
        
    return model, tokenizer

In [ ]:
def run_pplm_example(pretrained_model="gpt2-medium",
                     priming_sample="",
                     affect_weight=0.2,
                     knob=None,
                     uncond=False,
                     num_samples=1,
                     bag_of_words=None,
                     bag_of_words_affect=None,
                     discrim_weights=None,
                     discrim_meta=None,
                     class_label=-1,
                     length=100,
                     stepsize=0.02,
                     temperature=1.0,
                     top_k=10,
                     sample=True,
                     num_iterations=3,
                     grad_length=10000,
                     horizon_length=1,
                     window_length=0,
                     decay=False,
                     gamma=1.5,
                     gm_scale=0.9,
                     kl_scale=0.01,
                     seed=0,
                     no_cuda=False,
                     colorama=False,
                     verbosity='regular',
                     ):
    # set Random seed
    torch.manual_seed(seed)
    np.random.seed(seed)

    device = "cuda" if torch.cuda.is_available() and not no_cuda else "cpu"

    verbosity_level = VERBOSITY_LEVELS.get(verbosity.lower(), REGULAR)

    # load pretrained model and tokenizer
    model, tokenizer = load_model()
    model.to(device)
    model.eval()

    # Freeze GPT-2 weights
    for param in model.parameters():
        param.requires_grad = False

    if uncond:
        tokenized_cond_text = [tokenizer['BOS_None']]
    else:
        tokenized_cond_text = [tokenizer['BOS_None']]

    unpert_gen_tok_text, pert_gen_tok_texts, _, _ = full_text_generation(
        model=model,
        tokenizer=tokenizer,
        affect_weight=affect_weight,
        knob=knob,
        context=tokenized_cond_text,
        device=device,
        num_samples=num_samples,
        bag_of_words=bag_of_words,
        bag_of_words_affect=bag_of_words_affect,
        class_label=class_label,
        length=length,
        stepsize=stepsize,
        temperature=temperature,
        top_k=top_k,
        sample=sample,
        num_iterations=num_iterations,
        grad_length=grad_length,
        horizon_length=horizon_length,
        window_length=window_length,
        decay=decay,
        gamma=gamma,
        gm_scale=gm_scale,
        kl_scale=kl_scale,
        verbosity_level=verbosity_level
    )


In [ ]:
run_pplm_example(
                 affect_weight=1,  # it is the convergence rate of emotion loss, don't change it :-p
                 knob=knob,  # 0-1, play with it as much as you want
                 priming_sample=priming_sample,
                 num_samples=1,
                 bag_of_words_affect=emotion,
                 length=100,
                 stepsize=0.005,  # topic, emotion convergence rate
                 sample=True,
                 num_iterations=10,
                 window_length=5,
                 gamma=1.5,
                 gm_scale=0.95,
                 kl_scale=0.01,
                 verbosity='very_verbose'
            )